# Trying Another (Possible?) Code for Identifying Clusters

First, we import Acqu (which parses the events?) and Timepix (which decodes the data??), as well as numpy, ROOT, and plotting classes of ROOT which allow us to organize and graph the data.

In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas
from ipykernel import kernelapp as app

Welcome to JupyROOT 6.16/00


Next, we import the data that was collected and open that data file.


In [2]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


In [3]:
TimepixAData = []
TimepixBData = []

def plotTimepix():
    if(aq.epicsEvent==1):
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
        encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
        TimepixAData.append(Timepix.Decode(nHitsA,encodedA))
        TimepixBData.append(Timepix.Decode(nHitsB,encodedB))       
aq.runFunction(plotTimepix,0,50000)

I wasn't sure how to access the data the way it was structured, so I attempted to create a new array of data points with the ToA, x position, and y position, and sorted them by ToA.

In [21]:
ToAAData = []
ToAA = []

xAData = []
xA = []

yAData = []
yA = []

dataA = []

for i in range(len(TimepixAData)):
    ToAAData.append(TimepixAData[i]['ToA'])
    xAData.append(TimepixAData[i]['x'])
    yAData.append(TimepixAData[i]['y'])
    
for j in range(len(ToAAData)):
    for k in range(len(ToAAData[j])):
        ToAA.append(ToAAData[j][k])
        
for a in range(len(xAData)):
    for b in range(len(xAData[a])):
        xA.append(xAData[a][b])
        
for c in range(len(yAData)):
    for d in range(len(yAData[c])):
        yA.append(yAData[c][d])  
        
for z in range(len(ToAA)):
    dataA.append((ToAA[z],xA[z],yA[z]))

dataA.sort()

print(dataA)

###############################################

ToABData = []
ToAB = []

xBData = []
xB = []

yBData = []
yB = []

dataB = []

for i in range(len(TimepixBData)):
    ToABData.append(TimepixBData[i]['ToA'])
    xBData.append(TimepixBData[i]['x'])
    yBData.append(TimepixBData[i]['y'])
    
for j in range(len(ToABData)):
    for k in range(len(ToABData[j])):
        ToAB.append(ToABData[j][k])
        
for a in range(len(xBData)):
    for b in range(len(xBData[a])):
        xB.append(xBData[a][b])
        
for c in range(len(yBData)):
    for d in range(len(yBData[c])):
        yB.append(yBData[c][d])

for z in range(len(ToAB)):
    dataB.append((ToAB[z],xB[z],yB[z]))

dataB.sort()

[(147, 158, 207), (183, 220, 19), (184, 212, 7), (184, 213, 7), (185, 219, 19), (186, 213, 8), (304, 227, 107), (304, 227, 107), (308, 226, 107), (308, 226, 107), (387, 164, 223), (405, 196, 202), (407, 197, 202), (661, 225, 219), (661, 225, 219), (1576, 26, 89), (1576, 26, 89), (1577, 33, 89), (1577, 33, 89), (1577, 35, 92), (1577, 35, 92), (1577, 37, 95), (1577, 37, 95), (1577, 37, 97), (1577, 37, 97), (1577, 38, 97), (1577, 38, 97), (1578, 37, 94), (1578, 37, 94), (1579, 12, 54), (1579, 12, 54), (1739, 223, 14), (3090, 189, 119), (3090, 189, 119), (3435, 148, 169), (3435, 148, 169), (3436, 147, 169), (3436, 147, 169), (3576, 202, 133), (3576, 202, 133), (3716, 20, 111), (3716, 20, 111), (3717, 19, 111), (3717, 19, 111), (3718, 19, 112), (3718, 19, 112), (3725, 154, 35), (3725, 154, 36), (3727, 153, 36), (3864, 154, 220), (3867, 154, 219), (4009, 154, 121), (4009, 154, 121), (4010, 153, 121), (4010, 153, 121), (4084, 215, 167), (4085, 215, 168), (4165, 190, 228), (4165, 190, 228), (4

REMINDER: the 0th index is ToA, the 1st is x position, and the 2nd is y position

At some point we'll need to process/analyze the cluster we have (note that this currently only deals with the A detector's data):

In [40]:
def process_cluster():
    start_index = cl[0]                  # start index of the first hit in the cluster
    min_time = dataA[start_index][0]     # time of the first hit in the cluster
    #dtime                       # time difference?
    meanx = 0.0
    meany = 0.0
    
    # this next bit was a for loop in Ken's code
    n = start_index
    while(n<start_index+cn):             # go through each hit in the cluster
        if(dataA[n][0] < min_time):      # if we find a time smaller than the current min time
            min_time = dataA[n][0]       # reassign the smaller time to be the min time
        meanx+=dataA[n][1]               # add up all the x positions
        meany+=dataA[n][2]               # add up all the y positions
        n+=1                             # move to the next hit in the cluster
    
    if (cn!=0):
        meanx/=cn                        # calculate the mean x position
        meany/=cn                        # calculate the mean y position
    
    #print("meanx:",meanx, "meany:",meany, "min time:",min_time)
    
    
    tdpos.Fill(meanx,meany)
    
    
    
    
   # mult = Hist(100,0,100)
    #mult.Fill(nc)
    
  #  print(meanx, meany)
    
  #  c4 = ROOT.TCanvas("my4","The Canvas Title",1000,1000)
 #   c4.Divide(2,1)
 #   c4.cd(1)
  #  d2pos.Draw("colz")
   # c4.cd(2)
    #mult.Draw()
    #c4.Draw()
    
    

Now let's try to identify the clusters. Note that this currently only deals with the A detector's data.

In [39]:
cl = [None]*100          # a cluster can hold up to 100 hits
cn = 0                   # counts number of hits in current cluster
blen = 0                 # the length of the current tp3 buffer (?)
b = 0                    # the running (current?) buffer position
nc = 0                   # count the number of clusters found

ROOT.enableJSVis()
tdpos = Hist2D(100,0,300,100,0,300)
c4 = ROOT.TCanvas("my4","Canvas Title", 500,500)

blen = 1000#readTimepixBufferFromFile() - but this has no definition
#while(blen): # read each timpix buffer into tp3?
    
while(b<blen):        # for every buffer position in the buffer?
    cn = 0            # zero the cluster counter for the current buffer position
    cl[cn] = b        # assume the current position has the first hit in the cluster and save it#
    cn+=1             # increment the counter, since we've counted a hit
        
    # this next bit was a for loop in Ken's code
    c=b+1             # start at position c that is one above the current buffer position
    while((c<b+99) and (c<blen)):           # check all positions up to 99 away, as long as we're still in the buffer
        #print("c time:",dataA[c][0],"b time:",dataA[b][0])
        if(abs(dataA[c][0]- dataA[b][0] < 100)):   # if an event at c is within 100ns of that at the current buffer position
            cl[cn] = c                       # save this position to the cluster
            cn+=1                            # increment the counter, since we've counted a hit
        else:
            process_cluster()               # analyze the cluster we found
            b+=cn                           # move on beyond this cluster
            cn = 0                          # reset the counter for hits in the cluster
            cl = [None]*100                 # reset the array of cluster hits
            nc+=1                           # increment the number of clusters found
                
            if(c<b+99 and c<blen):          # if we're still in the loop
                cl[cn] = b                  # assume current position is the first in the new cluster
                cn+=1                       # increment the counter, since we've counted a hit
                
        c+=1 # move to the next position and go through this loop again
    b+=1

tdpos.Draw("colz")
c4.Draw()